# Covid-19 in Ireland
> Dashboard

- toc: false
- branch: master
- badges: true
- comments: false
- image: images/covid-19-ie-cases-per-m.png
- permalink: /covid-19-dashboard-ie/
- author: "Libre AI"
- categories: [ireland, covid19]

> Last updated on 2020-03-28 10:46 PM IST

---

<h1>Total Confirmed: <span style="color:red">2,415</span></h1>
<h1>Total Deaths: <span style="color:red">36</span></h1>
<h1>Total Recovered: <span style="color:green">5</span></h1>

---

> Important: **Starting on March 16th, 2020, Ireland has more #COVID19 cases per million (and growing quickly) than the most affected countries in Europe, Italy and Spain, at their early stages.** 

In [1]:
#hide
import altair as alt
import pandas as pd
import numpy as np

In [2]:
#hide
confirmed = pd.read_csv('data/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('data/time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('data/time_series_covid19_recovered_global.csv')

In [3]:
#hide

def dmy_to_isodate(dmy):
    m, d, y = dmy.split('/')
    return '{}-{:02d}-{:02d}'.format(2000 + int(y), int(m), int(d))
    
def get_count_data(df_all, country, start_count):
    df = df_all[df_all['Country/Region'] == country]
    counts = list(enumerate([(dmy_to_isodate(x[0]), x[1]) for x in list(zip(df.columns[4:], list(df.values[0][4:]))) if x[1] >= start_count]))
    days_since = [x for x,(_, _) in counts]
    dates = [x for _,(x, _)  in counts]
    freqs = [x for _,(_,x) in counts]
    return dates, freqs, days_since

def get_confirmed_df(df_all, country, start_count=10):
    dates, freqs, days_since = get_count_data(df_all, country, start_count)
    x = pd.DataFrame({
        'date': dates,
        'country': country,
        'cases': freqs,
        'new_cases': np.hstack((np.array([0]), np.diff(freqs))),
        'days_since_{}_cases'.format(start_count): days_since
    })
    return x

def get_deaths_df(df_all, country, start_count=0):
    dates, freqs, days_since = get_count_data(df_all, country, start_count)
    x = pd.DataFrame({
        'date': dates,
        'country': country,
        'deaths': freqs,
        'new_deaths': np.hstack((np.array([0]), np.diff(freqs))),
        'days_since_{}_deaths'.format(start_count): days_since
    })
    return x
    

In [4]:
#hide
cases_ie = get_confirmed_df(confirmed, 'Ireland')
cases_it = get_confirmed_df(confirmed, 'Italy')
cases_es = get_confirmed_df(confirmed, 'Spain')

In [6]:
#hide

#figures from https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population
population_in_m_ie = 4.9
population_in_m_it = 60.2
population_in_m_es = 47.1

In [7]:
#hide

source_ie = pd.DataFrame({
    'country': 'Ireland',
    'date': cases_ie['date'],
    'days_since_10_cases': cases_ie['days_since_10_cases'],
    'confirmed_cases': cases_ie['cases'],
    'confirmed_cases_per_million': np.round(cases_ie['cases']/population_in_m_ie),
    'new_cases': cases_ie['new_cases']
})

source_it = pd.DataFrame({
    'country': 'Italy',
    'date': cases_it['date'],
    'days_since_10_cases': cases_it['days_since_10_cases'],
    'confirmed_cases': cases_it['cases'],
    'confirmed_cases_per_million': np.round(cases_it['cases']/population_in_m_it),
    'new_cases': cases_it['new_cases']
})

source_es = pd.DataFrame({
    'country': 'Spain',
    'date': cases_es['date'],
    'days_since_10_cases': cases_es['days_since_10_cases'],
    'confirmed_cases': cases_es['cases'],
    'confirmed_cases_per_million': np.round(cases_es['cases']/population_in_m_es),
    'new_cases': cases_es['new_cases']
})


In [19]:
#hide

cases_ie_plot = alt.Chart(source_ie).mark_circle(
    color='green',
    opacity=0.7).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases:Q', scale=alt.Scale(type='sqrt')),
    size=alt.Size('new_cases:Q', title="Number of new cases in Ireland"),
    tooltip=['country', 'date:T', 'confirmed_cases', 'new_cases']
)#.interactive()

cases_ie_plot

alt.Chart(...)

In [9]:
#hide

cases_it_plot = alt.Chart(source_it).mark_line(
    color='blue',
    opacity=0.5,
    strokeDash=[4, 4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases', 'new_cases'],
    color=alt.Color('country', scale=alt.Scale(range=['blue'], domain=['Italy']), title="Country")
)



cases_it_plot

alt.Chart(...)

In [10]:
#hide

cases_es_plot = alt.Chart(source_es).mark_line(
    color='red',
    opacity=0.5,
    strokeDash=[4,4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases', 'new_cases'],
    color=alt.Color('country', scale=alt.Scale(range=['red'], domain=['Spain']), title="")
)

cases_es_plot

alt.Chart(...)

In [11]:
#hide
# # Number of persons tested positive for Covid-19 in Ireland
# The figure below shows the total number of persons who tested positive for Covid-19 (_cases_) in Ireland, Italy, and Spain. The `x` axis corresponds to the number of days since at least 10 cases were detected. This allow us to compare Ireland's current situation to the observed in Italy and Spain earlier. The comparison using Italy and Spain is because those countries are still in a critical situation fighting the pandemia, and no country would like to be the next one in such situation. 

# For Ireland, the size of the circles for each day denotes the number of new persons confirmed infected that day.

# **If we consider only the absolute numbers, we can see that the confirmed cases for Covid-19 in Ireland are below the ones observed in Italy and Spain at early stages. However, the populations of the three countries are very different and we have to account for that as well.**

In [12]:
#hide

alt.layer(cases_it_plot, cases_es_plot, cases_ie_plot).resolve_scale(
    color='independent',
    shape='independent'
)

alt.LayerChart(...)

---

In [13]:
#hide
# Number of Covid-19 cases per million

# Number of Covid-19 cases per million
The respective population of Ireland, Italy, and Spain is as follows (in millions) [\[source\]](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population):

- **Ireland** : 4.9M
- **Italy** : 60.2M
- **Spain** : 47.1M

If we plot a figure dividing the total number of cases reported daily by the population (in millions) we can find how many persons are infected per 1 million, which give us a normalized value we could use to understand the severity of the outbreak.

### The figure below shows that in fact Ireland has more cases per million than the most affected countries in Europe starting on March 16th, 2020, and growing quickly.

In [14]:
#hide

cases_per_million_ie_plot = alt.Chart(source_ie).mark_circle(
    color='green',
    opacity=0.7).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases_per_million:Q', scale=alt.Scale(type='sqrt')),
    size=alt.Size('new_cases:Q', title="Number of new cases in Ireland"),
    tooltip=['country', 'date:T', 'confirmed_cases:Q', 'confirmed_cases_per_million:Q', 'new_cases']
)#.interactive()

cases_per_million_ie_plot

alt.Chart(...)

In [15]:
#hide

cases_per_million_it_plot = alt.Chart(source_it).mark_line(
    color='blue',
    opacity=0.5,
    strokeDash=[4, 4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases_per_million:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases:Q', 'confirmed_cases_per_million:Q', 'new_cases'],
    color=alt.Color('country', scale=alt.Scale(range=['blue'], domain=['Italy']), title="Country")
)

cases_per_million_it_plot

alt.Chart(...)

In [16]:
#hide

cases_per_million_es_plot = alt.Chart(source_es).mark_line(
    color='red',
    opacity=0.5,
    strokeDash=[4,4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases_per_million:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases:Q', 'confirmed_cases_per_million:Q', 'new_cases'],
    color=alt.Color('country', scale=alt.Scale(range=['red'], domain=['Spain']), title="")
)

cases_per_million_es_plot

alt.Chart(...)

In [17]:
#hide_input

alt.layer(cases_per_million_ie_plot, cases_per_million_it_plot, cases_per_million_es_plot).properties(
    width=800, height=600
).resolve_scale(
    color='independent',
    shape='independent'
)

alt.LayerChart(...)

---
### Data Sources
- Government of Ireland [https://www.gov.ie/]
- 2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE [https://github.com/CSSEGISandData/COVID-19]
